In [1]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
sys.path.append(os.getcwd())
import tsa_utils
#import png

In [2]:
def build_model(self, data, labels, mode):
        print(data, labels, mode)
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=30
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        conv1 = tf.layers.conv3d(inputs=data, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv1")
        pool1 = tf.layers.max_pooling3d(inputs=conv1, pool_size=POOLSIZE1, strides=POOL_STRIDES, name="pool1")
        conv2 = tf.layers.conv3d(inputs=pool1, filters=FILTER_COUNT, kernel_size=(2,3,3), padding="same", strides=(1, 1, 1), name="conv2", activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling3d(inputs=conv2, pool_size=POOLSIZE1, strides=POOL_STRIDES, name="pool2")
        conv3 = tf.layers.conv3d(inputs=pool2, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv3")
        pool3 = tf.layers.max_pooling3d(inputs=conv3, pool_size=POOLSIZE1, strides=POOL_STRIDES, name="pool1")
        conv4 = tf.layers.conv3d(inputs=pool3, filters=FILTER_COUNT, kernel_size=(2,3,3), padding="same", strides=(1, 1, 1), name="conv4", activation=tf.nn.relu)
        pool4 = tf.layers.max_pooling3d(inputs=conv4, pool_size=POOLSIZE2, strides=POOL_STRIDES, name="pool2")
        
        flat_pool = tf.reshape(pool2, [BATCH_SIZE, 133056])
        flat_pool=tf.identity(flat_pool, name="flat_pool")

        logits = tf.layers.dense(inputs=flat_pool, units=2)
        logits =tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,2])

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            flat_labels = tf.reshape(labels, [BATCH_SIZE, 2])
            test_labels=tf.identity(flat_labels, name="labels")
            class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
            loss = tf.losses.softmax_cross_entropy(onehot_labels=test_labels, logits=logits, weights=class_weights)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.softmax(
              logits, name="softmax_tensor")}
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [6]:
tsa_classifier = tf.contrib.learn.Estimator(model_fn=build_model, 
                                            model_dir="/tmp/tsa_models/tsa1Zone2")
print(tsa_classifier.get_variable_names())
conv_filter = tsa_classifier.get_variable_value("dense/kernel")
print("CONV FILTER " +str(conv_filter))
print(tsa_classifier.get_variable_value("global_step"))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x86c9e50>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
['OptimizeLoss/learning_rate', 'conv1/bias', 'conv1/kernel', 'conv2/bias', 'conv2/kernel', 'conv3/bias', 'conv3/kernel', 'conv4/bias', 'conv4/kernel', 'dense/bias', 'dense/kernel', 'global_step']
CONV FILTER [[ -3.12634814e-03  -2.36896472e-03]
 [ -2.72637175e-04  -6.45036576e-03]
 [  5.66426788e-05   4.54946700e-03]
 ..., 
 [  3.52403196e-03   4.15461836e-03]
 [ -1.39239000e-03   6.20322861e-0

In [8]:

filters = tsa_classifier.get_variable_value("conv2d/kernel")

print(filters.shape)

x = filters[:,:,0,0]
#plt.imshow(x)
#plt.show()


# In[21]:

dir(tf.python_io)


NameError: name 'tsa_classifier' is not defined

In [29]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 

sys.path.append(os.getcwd())
import tsa_utils
import deep_cnn
import mini_cnn

image_df = pd.read_csv( './images/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()
image_df = image_df.groupby(["id"]).sum()
image_df.loc[image_df['Probability'] == 0, 'probability'] = 0
image_df.loc[image_df['Probability'] >= 1, 'probability'] = 1

In [30]:
image_df

,Probability,probability
id,,
00360f79fd6e02781457eda48f85da90,1,1.0
0043db5e8c819bffc15261b1f1ac5e42,3,1.0
0050492f92e22eed3474ae3a6fc907fa,3,1.0
006ec59fa59dd80a64c85347eef810c7,1,1.0
0097503ee9fa0606559c56458b281a08,1,1.0
011516ab0eca7cad7f5257672ddde70e,1,1.0
01941f33fd090ae5df8c95992c027862,3,1.0
01c08047f617de893bef104fb309203a,2,1.0
0240c8f1e89e855dcd8f1fa6b1e2b944,2,1.0
